In [3]:
import sys
sys.path.append('/Users/dhairya/cs projects/ideas 2025')
from main_functions import api_calling

In [5]:
api_calling.get_monthly_data('BAJFINANCE.BSE')

,close,volume
date,,
2025-06-13,9334.1504,501095
2025-05-30,9174.7500,654816
2025-04-30,8635.7000,999627
2025-03-28,8948.7500,941524
2025-02-28,8542.2500,747034
...,...,...
2005-06-30,280.0000,1834022
2005-05-31,230.6500,1891602
2005-04-29,183.1000,800983


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

risk_want = 0.5

tickers = ['BAJFINANCE.BSE', 'RELIANCE.BSE', 'HDFCBANK.BSE', 'TCS.BSE', 'INFY.BSE']
stock_data = {}

for ticker in tickers:
    df = api_calling.get_monthly_data(ticker)
    stock_data[ticker] = df['close']

returns = pd.DataFrame({ticker: data.pct_change() for ticker, data in stock_data.items()})
returns = returns.dropna()

mean_returns = returns.mean()
cov_matrix = returns.cov()

risk_free_rate = 0.06299

num_portfolios = 10000
results = []

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights = weights/np.sum(weights)
    
    portfolio_return = np.sum(mean_returns * weights) * 12
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 12, weights)))
    
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    results.append({'Return': portfolio_return, 
                   'Risk': portfolio_std,
                   'Sharpe': sharpe_ratio,
                   'Weights': weights})

results_df = pd.DataFrame(results)
max_sharpe_port = results_df.iloc[results_df['Sharpe'].idxmax()]
min_var_port = results_df.iloc[results_df['Risk'].idxmin()]

x_cml = np.linspace(0, max(results_df['Risk']), 100)
y_cml = risk_free_rate + (max_sharpe_port['Return'] - risk_free_rate) * x_cml / max_sharpe_port['Risk']

plt.figure(figsize=(10, 6))
plt.scatter(results_df['Risk'], results_df['Return'], c=results_df['Sharpe'], cmap='viridis', alpha=0.5)
plt.colorbar(label='Sharpe Ratio')
plt.plot(x_cml, y_cml, 'r--', label='Capital Market Line')
plt.scatter(max_sharpe_port['Risk'], max_sharpe_port['Return'], color='red', marker='*', s=200, label='Optimal Portfolio')
plt.scatter(min_var_port['Risk'], min_var_port['Return'], color='green', marker='*', s=200, label='Minimum Variance')
plt.plot(0, risk_free_rate, 'k*', markersize=15, label='Risk-Free Rate')
plt.xlabel('Risk (Standard Deviation)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier and Capital Market Line')
plt.legend()
plt.show()

print("\nOptimal Portfolio:")
print(f"Expected Return: {max_sharpe_port['Return']:.4f}")
print(f"Risk: {max_sharpe_port['Risk']:.4f}")
print(f"Sharpe Ratio: {max_sharpe_port['Sharpe']:.4f}")
print("\nOptimal Weights:")
for ticker, weight in zip(tickers, max_sharpe_port['Weights']):
    print(f"{ticker}: {weight:.4f}")


ValueError: Could not get monthly time series data for INFY.BSE. Response: {'Information': 'We have detected your API key as 8U4V3SZWG1XYKDJM and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}